<a href="https://colab.research.google.com/github/mjayashri/llm_from_scratch/blob/main/building_llm_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q ipdb
!pip install -q tqdm
!pip install -q sentencepiece
!pip install -q wandb

In [10]:
import os, sys
import ipdb
from tqdm import tqdm
from datetime import datetime
import platform, shutil
import requests, zipfile, io

#pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F

#tokenizer
import sentencepiece as spm

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

torch.cuda.empty_cache()

In [11]:
files_url = "https://ideami.com/llm_train"
print("Downloading files...")

response = requests.get(files_url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
zip_file.extractall(".")
print("Done!")

Done!


In [ ]:
# Architecture parameters
batch_size = 128
context = 512
embed_size = 384

n_layers = 7
n_heads = 7
BIAS = True

# Hyperparameters
lr = 3e-4
dropout = 0.05
weight_decay = 0.01
grad_clip = 1.0

# Training parameters
train_iters = 100000
eval_interval = 50
eval_iters = 10
